In [2]:
import netCDF4 as nc
import numpy as np
import datetime
import rasterio
import glob
import rioxarray
import xarray
import os
import glob

# List of NetCDF file paths
# nc_files = [r'etincr_monthly.nc']  # Add your file paths here
nc_files = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_inputs\*.nc')
output_folder = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_outputs\dry"
# Give reference image for profile
ref_img = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Yearly_outputs\Incomati_P_CHIRPS_yearly_avg.tif"
x =0
for i in nc_files:
    print(i, x)
    x += 1

# Ensure the output folder exists; create it if necessary
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# Loop through the NetCDF files
for nc_file in nc_files:
    nc_file = nc.Dataset(nc_file)
    time_var = nc_file.variables['time']
    units = time_var.units
    # Extract the reference date
    
    var_names = nc_file.variables.keys()
    print(var_names)

    # Specify the target variable names
    target_var_names = ['P', 'AET', 'Incremental_ET_M', 'Rainfall_ET_M', 'SRO_M' ]

    # Initialize the selected_variable variable as None
    selected_variable = None

    # Loop through the variable names and find the first match
    for var_name in var_names:
        if var_name in target_var_names:
            selected_variable = var_name
            break
    t1 = ['P', 'AET']
    if selected_variable in t1:
        # reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    else:
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    
    if selected_variable == 'P':
        file_name = "Incomati_Dry_Seasonal_Rainfall.tif"
    if selected_variable == 'AET':
        file_name = "Incomati_Dry_Seasonal_ET.tif"
    if selected_variable == 'Incremental_ET_M':
        file_name = "Incomati_Dry_Seasonal_Incremental_ET.tif"
    if selected_variable == 'Rainfall_ET_M':
        file_name = "Incomati_Dry_Seasonal_Rainfall_ET.tif"
    if selected_variable == 'SRO_M':
        file_name = "Incomati_Dry_Seasonal_Runoff.tif"
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    # Convert time values to datetime objects
    time_values = time_var[:]
    start_date = reference_date + datetime.timedelta(days=float(time_values[0]))
    print(start_date)
    end_date = reference_date + datetime.timedelta(days=float(time_values[-1]))
    print(end_date)
    # Find the indices corresponding to the first 6 months of every year
    start_month = 5
    end_month = 10
    start_month1 = 5
    end_month1 = 10
    indices = []
    for i, date in enumerate(time_values):
        dt = reference_date + datetime.timedelta(days=float(date))
        # print(dt.month)
        if ((start_month <= dt.month <= end_month) or (start_month1 <= dt.month <= end_month1)):
            indices.append(i)
        # Extract the data for the first 6 months of every year
        # target_var_names = ['P', 'ET', 'Incremental_ET_M', 'Rainfall_ET_M', 'SRO_M' ]
    data = nc_file.variables[selected_variable][indices, :, :]  # Replace 'your_variable_name' with the actual variable name
    # Calculate the sum of all bands
    sum_data = np.sum(data, axis=0)
    # Calculate the number of years
    if (start_month == start_month1 and end_month == end_month1):
        num_years = len(indices) / (end_month-start_month+1) # Divide by 2 to account for two sets of 6-month periods
    else:
        num_years = len(indices) / ((end_month-start_month+1) + (end_month1-start_month1+1))
        # print(num_years)
    # Calculate the average
    average_data = sum_data / num_years
    # Create a mask for values greater than the threshold
    mask = average_data > 1000000
    # Replace values greater than the threshold with NaN
    average_data[mask] = np.nan
    # Use os.path.dirname to get the directory path
    file_path = output_folder
#     file_name = file_name + "_avg.tif"

    # Concatenate the directory path and file name (without extension) to create the full file path
    full_file_path = os.path.join(file_path, file_name)
    print(full_file_path)
    
    dataset = rasterio.open(ref_img)
    output_profile = dataset.profile
    output_profile.update(count=1, dtype=rasterio.float32)

    with rasterio.open(full_file_path, 'w', **output_profile) as dst:
        dst.write(average_data.astype(rasterio.float32), 1)
    
    
# #     output_filename = r'average_output.tif'
#     with rasterio.open(full_file_path, 'w', driver='GTiff', height=average_data.shape[0], width=average_data.shape[1], count=1,
#                         dtype=average_data.dtype, crs='EPSG:4326') as dst:
#         dst.write(average_data, indexes=1)
        
    if selected_variable == 'P':
        file_name1 = "Incomati_Dry_Seasonal_Rainfall1.tif"
    if selected_variable == 'AET':
        file_name1 = "Incomati_Dry_Seasonal_ET1.tif"
    if selected_variable == 'Incremental_ET_M':
        file_name1 = "Incomati_Dry_Seasonal_Incremental_ET1.tif"
    if selected_variable == 'Rainfall_ET_M':
        file_name1 = "Incomati_Dry_Seasonal_Rainfall_ET1.tif"
    if selected_variable == 'SRO_M':
        file_name1 = "Incomati_Dry_Seasonal_Runoff1.tif"
    
    full_file_path1 = os.path.join(file_path, file_name1)
      
#     output_tif_path = r'average_output1.tif'
    # Threshold value
    threshold = 1000000
    # Open the input TIF file for reading
    with rasterio.open(full_file_path) as src:
        # Read the image data as a NumPy array
        image_data = src.read(1)
        # Create a mask for values greater than the threshold
        mask = image_data > threshold
        # Replace values greater than the threshold with NaN
        image_data[mask] = np.nan
        # Update metadata to handle NaN values
        #     profile = src.profile
        #     profile.update(nodata=np.nan)
        # Open the output TIF file for writing
        with rasterio.open(full_file_path1, 'w', **output_profile) as dst:
            dst.write(image_data.astype(rasterio.float32), 1)
#         with rasterio.open(full_file_path1, 'w', driver='GTiff', height=average_data.shape[0], width=average_data.shape[1], count=1,
#                         dtype=image_data.dtype, crs='EPSG:4326') as dst:
            # Write the modified data to the output TIF
#             dst.write(image_data, 1)
    # # print("Values greater than", threshold, "have been replaced with NaN in the output TIF file.")
    # Close the NetCDF file
    nc_file.close()
    print("Average GeoTIFF file created:", full_file_path1)


C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_inputs\Incomati_Incremental_ET_M_ETincr_monthly.nc 0
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_inputs\Incomati_P_CHIRPS_monthly.nc 1
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_inputs\Incomati_Rainfall_ET_M_ETrain_monthly.nc 2
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_inputs\Incomati_SRO_M_SRO_monthly.nc 3
dict_keys(['time', 'latitude', 'longitude', 'spatial_ref', 'Incremental_ET_M'])
2003-01-01 00:00:00
2021-12-01 00:00:00
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_outputs\dry\Incomati_Dry_Seasonal_Incremental_ET.tif
Average GeoTIFF file created: C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Incomati\Monthly_outputs\dry\Incomati_Dry_Seasonal_Incremental_ET1.tif
dict_keys(['time', 'latitude', 'longitude', 'spatial_ref', 'P'])
2003-01-01 00:00:00
2021-12-01 00:00:00
C:\Users

In [6]:
import netCDF4 as nc
import numpy as np
import datetime
import rasterio
import glob
import rioxarray
import xarray
import os
import glob

tif_files = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\*.tif')
x =0
for i in tif_files:
    print(i, x)
    x += 1

C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\ETincr_wet_avg.tif 0
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\ETincr_wet_avg1.tif 1
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\ETrain_wet_avg.tif 2
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\ETrain_wet_avg1.tif 3
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\Ghana_ETa_wet_avg.tif 4
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\P_wet_avg.tif 5
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\P_wet_avg1.tif 6
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\sro_wet_avg.tif 7
C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\sro_wet_avg1.tif 8


In [7]:
# Specify the paths to your TIFF files
P_Avg_tif = tif_files[6]
ET_Avg_tif = tif_files[4]

# file_path = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\for averaging\nc_files\output_montly_volta"

# Output TIFF file path for the difference
output_tif = file_path+"\Ghana_Water_yield_wet_avg.tif"
print(output_tif)

# Open the TIFF files using rasterio
with rasterio.open(P_Avg_tif) as p_avg_ds, rasterio.open(ET_Avg_tif) as et_avg_ds:
    # Read the data from the TIFF files as numpy arrays
    p_avg_array = p_avg_ds.read(1)
    et_avg_array = et_avg_ds.read(1)

    # Calculate the difference between the two arrays
    difference_array = p_avg_array - et_avg_array

#     # Copy the metadata from one of the input datasets
#     profile = p_avg_ds.profile
#     profile.update(count=1, dtype=rasterio.float32)

#     # Create the output TIFF file
#     with rasterio.open(output_tif, 'w', **profile) as output_ds:
#         # Write the difference array to the output dataset
#          output_ds.write(difference_array.astype(rasterio.float32), 1)
    with rasterio.open(output_tif, 'w', **output_profile) as dst:
            # Write the modified data to the output TIF
        dst.write(difference_array.astype(rasterio.float32), 1)

print(f"Difference saved to {output_tif}")

C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\Ghana_Water_yield_wet_avg.tif
Difference saved to C:\Users\t.perera\OneDrive - CGIAR\Documents\Continential\Ghana\netcdfs\output_monthly\wet\Ghana_Water_yield_wet_avg.tif


In [ ]:
import netCDF4 as nc

# Open the NetCDF file for reading
ncfile = nc.Dataset('etincr_monthly.nc', 'r')

# Extract the reference date attribute
reference_date_str = ncfile.getncattr('reference_date')

# Close the NetCDF file
ncfile.close()

# Parse the reference date string into a datetime object
reference_date = datetime.datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M')

print(reference_date)
